# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_correlation(): 
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nhldict  = {'New York Rangers':['New York City'],
                'New York Islanders':['New York City'],
                'New Jersey Devils':['New York City'],
                'Los Angeles Kings':['Los Angeles'],
                'Anaheim Ducks':['Los Angeles'],
                'San Jose Sharks':['San Francisco Bay Area'],
                'Chicago Blackhawks':['Chicago'],
                'Dallas Stars':['Dallas–Fort Worth'],
                'Washington Capitals':['Washington, D.C.'],
                'Philadelphia Flyers':['Philadelphia'],
                'Boston Bruins':['Boston'],
                'Minnesota Wild':['Minneapolis–Saint Paul'],
                'Colorado Avalanche':['Denver'],
                'Florida Panthers':['Miami–Fort Lauderdale'],
                'Arizona Coyotes':['Phoenix'],
                'Detroit Red Wings':['Detroit'],
                'Toronto Maple Leafs':['Toronto'],
                'Tampa Bay Lightning':['Tampa Bay Area'],
                'Pittsburgh Penguins':['Pittsburgh'],
                'St. Louis Blues':['St. Louis'],
                'Nashville Predators':['Nashville'],
                'Buffalo Sabres':['Buffalo'],
                'Montreal Canadiens':['Montreal'],
                'Vancouver Canucks':['Vancouver'],
                'Columbus Blue Jackets':['Columbus'],
                'Calgary Flames':['Calgary'],
                'Ottawa Senators':['Ottawa'],
                'Edmonton Oilers':['Edmonton'],
                'Winnipeg Jets':['Winnipeg'],
                'Vegas Golden Knights':['Las Vegas'],
                'Carolina Hurricanes':['Raleigh']
               }

        #### load and clean NHL team W/L data #####
    nhl2018 = nhl_df[nhl_df['year']== 2018]    #filter for 2018 data only
    nhl2018 = nhl2018[nhl2018['team'].str.contains('Division') == False]   #remove rows with division names
    nhl2018['team'] = nhl2018['team'].replace('\*', '', regex=True)  #remove asterisks from team names
    nhl2018 = nhl2018.set_index(['team']) 


        #### Add region for each team  in nhl2108 df#####

    nhlregdf = pd.DataFrame.from_dict(nhldict, orient = 'index', columns=['Metropolitan area'])
    nhlregdf.index.name='team'
    df2 = nhl2018.join(nhlregdf, how='outer')


        #### load and clean sports city wiki data #####
    cities = cities.replace('\[\S+\s\d+\]','',regex=True)  #remove notes
    cities = cities.replace('—',np.nan,regex=True)  #replace dashes with NaN
    cities = cities.replace('',np.nan,regex=True)  #replace blanks with NaN
    cities.rename(columns={'Population (2016 est.)[8]' : 'Population'}, inplace=True)

    cities = cities[['Metropolitan area','Population','NHL',]].dropna()

    cities = cities.merge(df2[['Metropolitan area','PTS%']], on='Metropolitan area')
    cities = cities.apply(pd.to_numeric, errors='ignore', downcast='float')

    cities =cities.groupby(
        ['Metropolitan area']
    ).agg(
        {
            'Population': 'mean',
            'PTS%': 'mean'
        }
    )


    population_by_region = cities['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = cities['PTS%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    corr, pval=stats.pearsonr(population_by_region, win_loss_by_region)
    return corr


In [2]:
nhl_correlation()

0.0067270688824815905

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


######## function code starts here ##############################################

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

nhldict  = {'New York Rangers':['New York City'],
            'New York Islanders':['New York City'],
            'New Jersey Devils':['New York City'],
            'Los Angeles Kings':['Los Angeles'],
            'Anaheim Ducks':['Los Angeles'],
            'San Jose Sharks':['San Francisco Bay Area'],
            'Chicago Blackhawks':['Chicago'],
            'Dallas Stars':['Dallas–Fort Worth'],
            'Washington Capitals':['Washington, D.C.'],
            'Philadelphia Flyers':['Philadelphia'],
            'Boston Bruins':['Boston'],
            'Minnesota Wild':['Minneapolis–Saint Paul'],
            'Colorado Avalanche':['Denver'],
            'Florida Panthers':['Miami–Fort Lauderdale'],
            'Arizona Coyotes':['Phoenix'],
            'Detroit Red Wings':['Detroit'],
            'Toronto Maple Leafs':['Toronto'],
            'Tampa Bay Lightning':['Tampa Bay Area'],
            'Pittsburgh Penguins':['Pittsburgh'],
            'St. Louis Blues':['St. Louis'],
            'Nashville Predators':['Nashville'],
            'Buffalo Sabres':['Buffalo'],
            'Montreal Canadiens':['Montreal'],
            'Vancouver Canucks':['Vancouver'],
            'Columbus Blue Jackets':['Columbus'],
            'Calgary Flames':['Calgary'],
            'Ottawa Senators':['Ottawa'],
            'Edmonton Oilers':['Edmonton'],
            'Winnipeg Jets':['Winnipeg'],
            'Vegas Golden Knights':['Las Vegas'],
            'Carolina Hurricanes':['Raleigh']
           }

    #### load and clean NHL team W/L data #####
nhl2018 = nhl_df[nhl_df['year']== 2018]    #filter for 2018 data only
nhl2018 = nhl2018[nhl2018['team'].str.contains('Division') == False]   #remove rows with division names
nhl2018['team'] = nhl2018['team'].replace('\*', '', regex=True)  #remove asterisks from team names
nhl2018 = nhl2018.set_index(['team']) 
   
    
    #### Add region for each team  in nhl2108 df#####

nhlregdf = pd.DataFrame.from_dict(nhldict, orient = 'index', columns=['Metropolitan area'])
nhlregdf.index.name='team'
df2 = nhl2018.join(nhlregdf, how='outer')


    #### load and clean sports city wiki data #####
cities = cities.replace('\[\S+\s\d+\]','',regex=True)  #remove notes
cities = cities.replace('—',np.nan,regex=True)  #replace dashes with NaN
cities = cities.replace('',np.nan,regex=True)  #replace blanks with NaN
cities.rename(columns={'Population (2016 est.)[8]' : 'Population'}, inplace=True)

cities = cities[['Metropolitan area','Population','NHL',]].dropna()

cities = cities.merge(df2[['Metropolitan area','PTS%']], on='Metropolitan area')
cities = cities.apply(pd.to_numeric, errors='ignore', downcast='float')

cities =cities.groupby(
    ['Metropolitan area']
).agg(
    {
        'Population': 'mean',
        'PTS%': 'mean'
    }
)


population_by_region = cities['Population'] # pass in metropolitan area population from cities
win_loss_by_region = cities['PTS%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
#stats.pearsonr(population_by_region, win_loss_by_region)
print(cities.apply(lambda row: row["PTS%"], axis=1))
len(cities)

Metropolitan area
Boston                    0.683000
Buffalo                   0.378000
Calgary                   0.512000
Chicago                   0.463000
Columbus                  0.591000
Dallas–Fort Worth         0.561000
Denver                    0.579000
Detroit                   0.445000
Edmonton                  0.476000
Las Vegas                 0.665000
Los Angeles               0.607000
Miami–Fort Lauderdale     0.585000
Minneapolis–Saint Paul    0.616000
Montreal                  0.433000
Nashville                 0.713000
New York City             0.516333
Ottawa                    0.409000
Philadelphia              0.598000
Phoenix                   0.427000
Pittsburgh                0.610000
Raleigh                   0.506000
San Francisco Bay Area    0.610000
St. Louis                 0.573000
Tampa Bay Area            0.689000
Toronto                   0.640000
Vancouver                 0.445000
Washington, D.C.          0.640000
Winnipeg                  0.695000
dt

28

In [4]:
for i in cities:
    print(i).line()

Population


AttributeError: 'NoneType' object has no attribute 'line'

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nba_correlation():
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nbadict  = {'New York Knicks':['New York City'],
                'Los Angeles Lakers':['Los Angeles'],
                'Los Angeles Clippers':['Los Angeles'],
                'Golden State Warriors':['San Francisco Bay Area'],
                'Chicago Bulls':['Chicago'],
                'Dallas Mavericks':['Dallas–Fort Worth'],
                'Washington Wizards':['Washington, D.C.'],
                'Philadelphia 76ers':['Philadelphia'],
                'Boston Celtics':['Boston'],
                'Minnesota Timberwolves':['Minneapolis–Saint Paul'],
                'Denver Nuggets':['Denver'],
                'Miami Heat':['Miami–Fort Lauderdale'],
                'Phoenix Suns':['Phoenix'],
                'Detroit Pistons':['Detroit'],
                'Toronto Raptors':['Toronto'],
                'Houston Rockets':['Houston'],
                'Atlanta Hawks':['Atlanta'],
                'Cleveland Cavaliers':['Cleveland'],
                'Charlotte Hornets':['Charlotte'],
                'Indiana Pacers':['Indianapolis'],
                'Milwaukee Bucks':['Milwaukee'],
                'New Orleans Pelicans':['New Orleans'],
                'Orlando Magic':['Orlando'],
                'Portland Trail Blazers':['Portland'],
                'Utah Jazz':['Salt Lake City'],
                'San Antonio Spurs':['San Antonio'],
                'Sacramento Kings':['Sacramento'],
                'Oklahoma City Thunder':['Oklahoma City'],
                'Memphis Grizzlies':['Memphis']
               }


        #### load and clean NHL team W/L data #####
    nba2018 = nba_df[nba_df['year']== 2018]    #filter for 2018 data only
    nba2018['team'] = nba2018['team'].replace('\*', '', regex=True)  #remove asterisks from team names
    nba2018['team'] = nba2018['team'].replace('\s\(\S+', '', regex=True)  #remove parenthesis notes from team names
    nba2018 = nba2018.set_index(['team']) 


         #### Add region for each team  in nba2018 df#####
    nbaregdf = pd.DataFrame.from_dict(nbadict, orient = 'index', columns=['Metropolitan area'])
    nbaregdf.index.name='team'
    df2 = nba2018.join(nbaregdf, how='outer')

        #### load and clean sports city wiki data #####
    cities = cities.replace('\[\S+\s\d+\]','',regex=True)  #remove notes
    cities = cities.replace('—',np.nan,regex=True)  #replace dashes with NaN
    cities = cities.replace('',np.nan,regex=True)  #replace blanks with NaN
    cities.rename(columns={'Population (2016 est.)[8]' : 'Population'}, inplace=True)

    cities = cities[['Metropolitan area','Population','NBA',]].dropna()

    cities = cities.merge(df2[['Metropolitan area','W/L%']], on='Metropolitan area')
    cities = cities.apply(pd.to_numeric, errors='ignore', downcast='float')

    cities =cities.groupby(
        ['Metropolitan area']
    ).agg(
        {
            'Population': 'mean',
            'W/L%': 'mean'
        }
    )


    population_by_region = cities['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = cities['W/L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    corr, pval=stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

In [ ]:
nba_correlation()

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


##################  code for nba corr function #################
nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

nbadict  = {'New York Knicks':['New York City'],
            'Los Angeles Lakers':['Los Angeles'],
            'Los Angeles Clippers':['Los Angeles'],
            'Golden State Warriors':['San Francisco Bay Area'],
            'Chicago Bulls':['Chicago'],
            'Dallas Mavericks':['Dallas–Fort Worth'],
            'Washington Wizards':['Washington, D.C.'],
            'Philadelphia 76ers':['Philadelphia'],
            'Boston Celtics':['Boston'],
            'Minnesota Timberwolves':['Minneapolis–Saint Paul'],
            'Denver Nuggets':['Denver'],
            'Miami Heat':['Miami–Fort Lauderdale'],
            'Phoenix Suns':['Phoenix'],
            'Detroit Pistons':['Detroit'],
            'Toronto Raptors':['Toronto'],
            'Houston Rockets':['Houston'],
            'Atlanta Hawks':['Atlanta'],
            'Cleveland Cavaliers':['Cleveland'],
            'Charlotte Hornets':['Charlotte'],
            'Indiana Pacers':['Indianapolis'],
            'Milwaukee Bucks':['Milwaukee'],
            'New Orleans Pelicans':['New Orleans'],
            'Orlando Magic':['Orlando'],
            'Portland Trail Blazers':['Portland'],
            'Utah Jazz':['Salt Lake City'],
            'San Antonio Spurs':['San Antonio'],
            'Sacramento Kings':['Sacramento'],
            'Oklahoma City Thunder':['Oklahoma City'],
            'Memphis Grizzlies':['Memphis']
           }


    #### load and clean NHL team W/L data #####
nba2018 = nba_df[nba_df['year']== 2018]    #filter for 2018 data only
nba2018['team'] = nba2018['team'].replace('\*', '', regex=True)  #remove asterisks from team names
nba2018['team'] = nba2018['team'].replace('\s\(\S+', '', regex=True)  #remove parenthesis notes from team names
nba2018 = nba2018.set_index(['team']) 
   
    
     #### Add region for each team  in nba2018 df#####
nbaregdf = pd.DataFrame.from_dict(nbadict, orient = 'index', columns=['Metropolitan area'])
nbaregdf.index.name='team'
df2 = nba2018.join(nbaregdf, how='outer')

    #### load and clean sports city wiki data #####
cities = cities.replace('\[\S+\s\d+\]','',regex=True)  #remove notes
cities = cities.replace('—',np.nan,regex=True)  #replace dashes with NaN
cities = cities.replace('',np.nan,regex=True)  #replace blanks with NaN
cities.rename(columns={'Population (2016 est.)[8]' : 'Population'}, inplace=True)

cities = cities[['Metropolitan area','Population','NBA',]].dropna()

cities = cities.merge(df2[['Metropolitan area','W/L%']], on='Metropolitan area')
cities = cities.apply(pd.to_numeric, errors='ignore', downcast='float')

cities =cities.groupby(
    ['Metropolitan area']
).agg(
    {
        'Population': 'mean',
        'W/L%': 'mean'
    }
)


population_by_region = cities['Population'] # pass in metropolitan area population from cities
win_loss_by_region = cities['W/L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

stats.pearsonr(population_by_region, win_loss_by_region)

#cities
#nba2018
#nbaregdf
len(population_by_region)
#len(win_loss_by_region)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def mlb_correlation(): 
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    mlbdict  = {'New York Yankees':['New York City'],
                'New York Mets':['New York City'],
                'Los Angeles Angels':['Los Angeles'],
                'Los Angeles Dodgers':['Los Angeles'],
                'San Francisco Giants':['San Francisco Bay Area'],
                'Oakland Athletics':['San Francisco Bay Area'],
                'Chicago Cubs':['Chicago'],
                'Chicago White Sox':['Chicago'],
                'Texas Rangers':['Dallas–Fort Worth'],
                'Washington Nationals':['Washington, D.C.'],
                'Philadelphia Phillies':['Philadelphia'],
                'Boston Red Sox':['Boston'],
                'Minnesota Twins':['Minneapolis–Saint Paul'],
                'Colorado Rockies':['Denver'],
                'Miami Marlins':['Miami–Fort Lauderdale'],
                'Arizona Diamondbacks':['Phoenix'],
                'Detroit Tigers':['Detroit'],
                'Toronto Blue Jays':['Toronto'],
                'Houston Astros':['Houston'],
                'Atlanta Braves':['Atlanta'],
                'Tampa Bay Rays':['Tampa Bay Area'],
                'Pittsburgh Pirates':['Pittsburgh'],
                'Cleveland Indians':['Cleveland'],
                'Seattle Mariners':['Seattle'],
                'Cincinnati Reds':['Cincinnati'],
                'Kansas City Royals':['Kansas City'],
                'St. Louis Cardinals':['St. Louis'],
                'Baltimore Orioles':['Baltimore'],
                'Milwaukee Brewers':['Milwaukee'],
                'San Diego Padres':['San Diego']
               }


        #### load and clean NHL team W/L data #####
    mlb2018 = mlb_df[mlb_df['year']== 2018]    #filter for 2018 data only
    mlb2018 = mlb2018.set_index(['team']) 


         #### Add region for each team  in nba2018 df#####
    mlbregdf = pd.DataFrame.from_dict(mlbdict, orient = 'index', columns=['Metropolitan area'])
    mlbregdf.index.name='team'
    df2 = mlb2018.join(mlbregdf, how='outer')

        #### load and clean sports city wiki data #####
    cities = cities.replace('\[\S+\s\d+\]','',regex=True)  #remove notes
    cities = cities.replace('—',np.nan,regex=True)  #replace dashes with NaN
    cities = cities.replace('',np.nan,regex=True)  #replace blanks with NaN
    cities.rename(columns={'Population (2016 est.)[8]' : 'Population'}, inplace=True)

    cities = cities[['Metropolitan area','Population','MLB',]].dropna()

    cities = cities.merge(df2[['Metropolitan area','W-L%']], on='Metropolitan area')
    cities = cities.apply(pd.to_numeric, errors='ignore', downcast='float')

    cities =cities.groupby(
        ['Metropolitan area']
    ).agg(
        {
            'Population': 'mean',
            'W-L%': 'mean'
        }
    )


    population_by_region = cities['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = cities['W-L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    corr, pval=stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

In [ ]:
cities

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
##################  code for mlb corr function #################

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

mlbdict  = {'New York Yankees':['New York City'],
            'New York Mets':['New York City'],
            'Los Angeles Angels':['Los Angeles'],
            'Los Angeles Dodgers':['Los Angeles'],
            'San Francisco Giants':['San Francisco Bay Area'],
            'Oakland Athletics':['San Francisco Bay Area'],
            'Chicago Cubs':['Chicago'],
            'Chicago White Sox':['Chicago'],
            'Texas Rangers':['Dallas–Fort Worth'],
            'Washington Nationals':['Washington, D.C.'],
            'Philadelphia Phillies':['Philadelphia'],
            'Boston Red Sox':['Boston'],
            'Minnesota Twins':['Minneapolis–Saint Paul'],
            'Colorado Rockies':['Denver'],
            'Miami Marlins':['Miami–Fort Lauderdale'],
            'Arizona Diamondbacks':['Phoenix'],
            'Detroit Tigers':['Detroit'],
            'Toronto Blue Jays':['Toronto'],
            'Houston Astros':['Houston'],
            'Atlanta Braves':['Atlanta'],
            'Tampa Bay Rays':['Tampa Bay Area'],
            'Pittsburgh Pirates':['Pittsburgh'],
            'Cleveland Indians':['Cleveland'],
            'Seattle Mariners':['Seattle'],
            'Cincinnati Reds':['Cincinnati'],
            'Kansas City Royals':['Kansas City'],
            'St. Louis Cardinals':['St. Louis'],
            'Baltimore Orioles':['Baltimore'],
            'Milwaukee Brewers':['Milwaukee'],
            'San Diego Padres':['San Diego']
           }


    #### load and clean NHL team W/L data #####
mlb2018 = mlb_df[mlb_df['year']== 2018]    #filter for 2018 data only
mlb2018 = mlb2018.set_index(['team']) 
   
    
     #### Add region for each team  in nba2018 df#####
mlbregdf = pd.DataFrame.from_dict(mlbdict, orient = 'index', columns=['Metropolitan area'])
mlbregdf.index.name='team'
df2 = mlb2018.join(mlbregdf, how='outer')

    #### load and clean sports city wiki data #####
cities = cities.replace('\[\S+\s\d+\]','',regex=True)  #remove notes
cities = cities.replace('—',np.nan,regex=True)  #replace dashes with NaN
cities = cities.replace('',np.nan,regex=True)  #replace blanks with NaN
cities.rename(columns={'Population (2016 est.)[8]' : 'Population'}, inplace=True)

cities = cities[['Metropolitan area','Population','MLB',]].dropna()

cities = cities.merge(df2[['Metropolitan area','W-L%']], on='Metropolitan area')
cities = cities.apply(pd.to_numeric, errors='ignore', downcast='float')

cities =cities.groupby(
    ['Metropolitan area']
).agg(
    {
        'Population': 'mean',
        'W-L%': 'mean'
    }
)


population_by_region = cities['Population'] # pass in metropolitan area population from cities
win_loss_by_region = cities['W-L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

stats.pearsonr(population_by_region, win_loss_by_region)

cities
#len(mlb2018)
#mlbregdf
#df2
#len(population_by_region)
#len(win_loss_by_region)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nfl_correlation(): 
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nfldict  = {'New York Giants':['New York City'],
                'New York Jets':['New York City'],
                'Los Angeles Rams':['Los Angeles'],
                'Los Angeles Chargers':['Los Angeles'],
                'San Francisco 49ers':['San Francisco Bay Area'],
                'Oakland Raiders':['San Francisco Bay Area'],
                'Chicago Bears':['Chicago'],
                'Dallas Cowboys':['Dallas–Fort Worth'],
                'Washington Redskins':['Washington, D.C.'],
                'Philadelphia Eagles':['Philadelphia'],
                'New England Patriots':['Boston'],
                'Minnesota Vikings':['Minneapolis–Saint Paul'],
                'Denver Broncos':['Denver'],
                'Miami Dolphins':['Miami–Fort Lauderdale'],
                'Arizona Cardinals':['Phoenix'],
                'Detroit Lions':['Detroit'],
                'Houston Texans':['Houston'],
                'Atlanta Falcons':['Atlanta'],
                'Tampa Bay Buccaneers':['Tampa Bay Area'],
                'Pittsburgh Steelers':['Pittsburgh'],
                'Cleveland Browns':['Cleveland'],
                'Seattle Seahawks':['Seattle'],
                'Cincinnati Bengals':['Cincinnati'],
                'Kansas City Chiefs':['Kansas City'],
                'Baltimore Ravens':['Baltimore'],
                'Carolina Panthers':['Charlotte'],
                'Indianapolis Colts':['Indianapolis'],
                'Tennessee Titans':['Nashville'],
                'New Orleans Saints':['New Orleans'],
                'Buffalo Bills':['Buffalo'],
                'Jacksonville Jaguars':['Jacksonville'],
                'Green Bay Packers':['Green Bay']
               }


        #### load and clean nfl team W/L data #####
    nfl2018 = nfl_df[nfl_df['year']== 2018]    #filter for 2018 data only
    nfl2018 = nfl2018[nfl2018['team'].str.contains('\wFC') == False]   #remove rows with division names
    nfl2018['team'] = nfl2018['team'].replace('\*', '', regex=True)  #remove asterisks from team names
    nfl2018['team'] = nfl2018['team'].replace('\+', '', regex=True)  #remove pluses from team names
    nfl2018 = nfl2018.set_index(['team']) 


         #### Add region for each team  in nfl2018 df#####
    nflregdf = pd.DataFrame.from_dict(nfldict, orient = 'index', columns=['Metropolitan area'])
    nflregdf.index.name='team'
    df2 = nfl2018.join(nflregdf, how='outer')

        #### load and clean sports city wiki data #####
    cities = cities.replace('\[\S+\s\d+\]','',regex=True)  #remove notes
    cities = cities.replace('—',np.nan,regex=True)  #replace dashes with NaN
    cities = cities.replace('',np.nan,regex=True)  #replace blanks with NaN
    cities.rename(columns={'Population (2016 est.)[8]' : 'Population'}, inplace=True)

    cities = cities[['Metropolitan area','Population','NFL',]].dropna()

    cities = cities.merge(df2[['Metropolitan area','W-L%']], on='Metropolitan area')
    cities = cities.apply(pd.to_numeric, errors='ignore', downcast='float')

    cities =cities.groupby(
        ['Metropolitan area']
    ).agg(
        {
            'Population': 'mean',
            'W-L%': 'mean'
        }
    )


    population_by_region = cities['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = cities['W-L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    corr, pval=stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

In [ ]:
nfl_correlation()

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
##################  code for nfl corr function #################
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

nfldict  = {'New York Giants':['New York City'],
            'New York Jets':['New York City'],
            'Los Angeles Rams':['Los Angeles'],
            'Los Angeles Chargers':['Los Angeles'],
            'San Francisco 49ers':['San Francisco Bay Area'],
            'Oakland Raiders':['San Francisco Bay Area'],
            'Chicago Bears':['Chicago'],
            'Dallas Cowboys':['Dallas–Fort Worth'],
            'Washington Redskins':['Washington, D.C.'],
            'Philadelphia Eagles':['Philadelphia'],
            'New England Patriots':['Boston'],
            'Minnesota Vikings':['Minneapolis–Saint Paul'],
            'Denver Broncos':['Denver'],
            'Miami Dolphins':['Miami–Fort Lauderdale'],
            'Arizona Cardinals':['Phoenix'],
            'Detroit Lions':['Detroit'],
            'Houston Texans':['Houston'],
            'Atlanta Falcons':['Atlanta'],
            'Tampa Bay Buccaneers':['Tampa Bay Area'],
            'Pittsburgh Steelers':['Pittsburgh'],
            'Cleveland Browns':['Cleveland'],
            'Seattle Seahawks':['Seattle'],
            'Cincinnati Bengals':['Cincinnati'],
            'Kansas City Chiefs':['Kansas City'],
            'Baltimore Ravens':['Baltimore'],
            'Carolina Panthers':['Charlotte'],
            'Indianapolis Colts':['Indianapolis'],
            'Tennessee Titans':['Nashville'],
            'New Orleans Saints':['New Orleans'],
            'Buffalo Bills':['Buffalo'],
            'Jacksonville Jaguars':['Jacksonville'],
            'Green Bay Packers':['Green Bay']
           }


    #### load and clean nfl team W/L data #####
nfl2018 = nfl_df[nfl_df['year']== 2018]    #filter for 2018 data only
nfl2018 = nfl2018[nfl2018['team'].str.contains('\wFC') == False]   #remove rows with division names
nfl2018['team'] = nfl2018['team'].replace('\*', '', regex=True)  #remove asterisks from team names
nfl2018['team'] = nfl2018['team'].replace('\+', '', regex=True)  #remove pluses from team names
nfl2018 = nfl2018.set_index(['team']) 
   
    
     #### Add region for each team  in nfl2018 df#####
nflregdf = pd.DataFrame.from_dict(nfldict, orient = 'index', columns=['Metropolitan area'])
nflregdf.index.name='team'
df2 = nfl2018.join(nflregdf, how='outer')

    #### load and clean sports city wiki data #####
cities = cities.replace('\[\S+\s\d+\]','',regex=True)  #remove notes
cities = cities.replace('—',np.nan,regex=True)  #replace dashes with NaN
cities = cities.replace('',np.nan,regex=True)  #replace blanks with NaN
cities.rename(columns={'Population (2016 est.)[8]' : 'Population'}, inplace=True)

cities = cities[['Metropolitan area','Population','NFL',]].dropna()

cities = cities.merge(df2[['Metropolitan area','W-L%']], on='Metropolitan area')
cities = cities.apply(pd.to_numeric, errors='ignore', downcast='float')

cities =cities.groupby(
    ['Metropolitan area']
).agg(
    {
        'Population': 'mean',
        'W-L%': 'mean'
    }
)


population_by_region = cities['Population'] # pass in metropolitan area population from cities
win_loss_by_region = cities['W-L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]


len(cities)
#nfl2018
df2
win_loss_by_region

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values